In [ ]:
pip install PySastrawi

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import sklearn
import keras
import pickle
import tensorflow.keras as keras
import collections
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.base import BaseEstimator, TransformerMixin
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1
from tensorflow.keras.regularizers import l2
from tensorflow.keras.regularizers import L1L2
from keras.optimizers import Adam, RMSprop, SGD
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import confusion_matrix
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
#from terbilang import Terbilang

#t = Terbilang()
#nltk.download('stopwords')
#nltk.download('all')

In [ ]:
#to print complete sentence
pd.options.display.max_colwidth = 500

In [ ]:
#kamus
kamusalay = pd.read_csv('new_kamusalay.csv', encoding =  'ISO-8859-1', names=['old','new'])
kamusalay.head()

,old,new
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [ ]:
file_path = "train_preprocess.tsv.txt"
df = pd.read_csv(file_path, delimiter='\t', names=["Tweet", "Sentimen"])
#print(type(df["data"]))
#print(df["data"])

In [ ]:
def cleaning_data(text):
    text = text.lower()
    text=re.sub('url', '', text)
    text = re.sub('url\S+', '', text)
    text=re.sub('http', '', text)
    text = re.sub(r'http\S+', '', text)
    #text = re.sub(r'&', 'dan', text)
    #text=re.sub(r'/', 'atau', text)
    #text = re.sub(r'&amp', 'dan', text)
    #text = re.sub(r'&lt;', '', text)
    #text = re.sub(r'&gt;', '', text)
    text = re.sub(r'%', ' persen', text)
    text = re.sub(r' {2,}', ' ', text)
    text=re.sub(r'(\\x(.){2})', '', text) #emoticon
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    return text


In [ ]:
#split the data
def convert(lst):
    return ''.join(lst).split()

In [ ]:
# check with kamus alay
def filter_data(text):
        result = ""
        tweets = text
        #print(tweets)
        temp=[]
        for tweet in tweets:
            try:
                trying = np.where(kamusalay['old']==tweet)[0][0]
                tweet = kamusalay['new'].iloc[trying]
            except:
                tweet = tweet
            #print(tweet)
            temp.append(tweet)
            result = result + " " + tweet
        return result

In [ ]:
df['ReviewTweet'] = df['Tweet'].apply(cleaning_data)
df['ReviewTweet']=df['ReviewTweet'].apply(convert)
df['ReviewTweet']=df['ReviewTweet'].apply(filter_data)

In [ ]:
df.head()

,Tweet,Sentimen,ReviewTweet
0,"warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung . tahu berkualitas , dipadu keahlian memasak , dipadu kretivitas , jadilah warung yang menyajikan menu utama berbahan tahu , ditambah menu umum lain seperti ayam . semuanya selera indonesia . harga cukup terjangkau . jangan lewatkan tahu bletoka nya , tidak kalah dengan yang asli dari tegal !",positive,warung ini dimiliki oleh pengusaha pabrik tahu yang sudah puluhan tahun terkenal membuat tahu putih di bandung tahu berkualitas dipadu keahlian memasak dipadu kretivitas jadilah warung yang menyajikan menu utama berbahan tahu ditambah menu umum lain seperti ayam semuanya selera indonesia harga cukup terjangkau jangan melewati tahu bletoka nya tidak kalah dengan yang asli dari tegal
1,mohon ulama lurus dan k212 mmbri hujjah partai apa yang harus diwlh agar suara islam tidak pecah-pecah,neutral,mohon ulama lurus dan k212 mmbri hujjah partai apa yang harus diwlh agar suara islam tidak pecah pecah
2,"lokasi strategis di jalan sumatera bandung . tempat nya nyaman terutama sofa di lantai 2 . paella nya enak , sangat pas dimakan dengan minum bir dingin . appetiser nya juga enak-enak .",positive,lokasi strategis di jalan sumatra bandung tempat nya nyaman terutama sofa di lantai 2 paella nya enak sangat pas dimakan dengan minum bir dingin appetiser nya juga enak enak
3,betapa bahagia nya diri ini saat unboxing paket dan barang nya bagus ! menetapkan beli lagi !,positive,betapa bahagia nya diri ini saat unboxing paket dan barang nya bagus menetapkan beli lagi
4,"duh . jadi mahasiswa jangan sombong dong . kasih kartu kuning segala . belajar dulu yang baik , tidak usahlah ikut-ikut politik . nanti sudah selesai kuliah nya mau ikut politik juga tidak telat . dasar mahasiswa .",negative,aduh jadi mahasiswa jangan sombong dong kasih kartu kuning segala belajar dulu yang baik tidak usahlah ikut ikut politik nanti sudah selesai kuliah nya mau ikut politik juga tidak telat dasar mahasiswa


In [ ]:
#use Bag of Words (BOW)
data_preprocessed = df['ReviewTweet'].tolist()

vectorizer = CountVectorizer()
vectorizer.fit(data_preprocessed)
X = vectorizer.transform(data_preprocessed).toarray()

(16235,)

In [ ]:
#to save the model
pickle.dump(vectorizer, open('featureBOWKA.p','wb'))

In [ ]:
classes = df['Sentimen']

In [ ]:
# Split the data (with composition data train 80%, data test 20%)
X_train, X_test, y_train, y_test = train_test_split(X, classes, test_size = 0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8800, 16235) (8800,)
(2200, 16235) (2200,)


In [ ]:
model = MLPClassifier(learning_rate='constant', learning_rate_init=0.001)
model = model.fit(X_train, y_train)

In [ ]:
#to save the model
pickle.dump(model, open('model-sentimentBOWKA.p','wb'))

In [ ]:
test = model.predict(X_test)

print(classification_report(y_test, test))

              precision    recall  f1-score   support

    negative       0.78      0.79      0.79       666
     neutral       0.75      0.67      0.71       214
    positive       0.89      0.90      0.90      1320

    accuracy                           0.85      2200
   macro avg       0.81      0.79      0.80      2200
weighted avg       0.85      0.85      0.85      2200



In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []

y = classes

for iteration, data in enumerate(kf.split(X), start=1):
  data_train = X[data[0]]
  target_train = y[data[0]]

  data_test = X[data[1]]
  target_test = y[data[1]]

  clf = MLPClassifier(learning_rate='constant', learning_rate_init=0.001)
  clf.fit(data_train, target_train)

  preds = clf.predict(data_test)

  accuracy = accuracy_score(target_test, preds)

  print("Training ke-", iteration)
  print(classification_report(target_test, preds))
  print("_______________________________________")

  accuracies.append(accuracy)

average_accuracy = np.mean(accuracies)

print()
print()
print()
print("Average accuracy :", average_accuracy)

Training ke- 1
              precision    recall  f1-score   support

    negative       0.78      0.78      0.78       680
     neutral       0.76      0.62      0.68       239
    positive       0.87      0.90      0.89      1281

    accuracy                           0.83      2200
   macro avg       0.80      0.77      0.78      2200
weighted avg       0.83      0.83      0.83      2200

_______________________________________
Training ke- 2
              precision    recall  f1-score   support

    negative       0.81      0.78      0.80       706
     neutral       0.75      0.73      0.74       220
    positive       0.89      0.91      0.90      1274

    accuracy                           0.85      2200
   macro avg       0.82      0.81      0.81      2200
weighted avg       0.85      0.85      0.85      2200

_______________________________________
Training ke- 3
              precision    recall  f1-score   support

    negative       0.79      0.79      0.79       682
    

In [ ]:
# prediction
original_text =  '''
saya suka makan di restoran itu karena makanannya enak enak
'''
# Feature Extraction
text = vectorizer.transform([cleaning_data(original_text)])

#print(text) #0, 14410

# predict the sentiment
result = model.predict(text)[0]
print("Sentiment:")
print()
print(result)


**Confusion Matrix**

In [ ]:
classes = df['Sentimen']
cm = confusion_matrix(y_test, test, labels=classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot()
plt.show()